In [1]:
import pandas as pd
import numpy as np
from p_tqdm import p_map

In [2]:
from const import DATASETS,SETTINGS

In [130]:
conf = {
 'apiurl': 'http://www.nomisweb.co.uk/api/v01/dataset/{dataset}.data.csv?date=latest&geography={geocode}&{cell}={cells}&measures=20100&select={selection}&uid={key}{query_misc}',
 'apikey': '0x3cfb19ead752b37bb90da0eb3a0fe78baa9fa055',
#  'outfile': './data.h5'
}

area = {#geocode
    'lad':'TYPE464'
}





# for areatype in area:
areatype = 'lad'

db_file = f'./{areatype}.sqlite'
import sqlite3
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))
conn = sqlite3.connect(db_file)
# cur = conn.cursor()

## Generate a area and total tally

In [108]:

pop = filter(lambda x:x['code'] == 'population',DATASETS).__next__()
sources = pop['sources']
df = False

for src in sources:

    src = src.copy()
    src['cells'] = ','.join(map(str,src['cells']))
    src['selection'] = ','.join(map(str,src['selection']))
    src['key'] = conf['apikey']
    if not hasattr(src,'query_misc'):
        src['query_misc']=''
    url = conf['apiurl'].format(**src)
    print(url)
    if type(df)!= bool:
        df = df.append(pd.read_csv(url),)
    else:
        df = pd.read_csv(url)
        
## on the assumption that geography codes for each year are different
population = df.groupby('GEOGRAPHY_CODE').first()['OBS_VALUE']

p_ref = pd.DataFrame(population.index).reset_index()
p_ref.columns = ['GEOGRAPHY_CODE','AREACD']
p_ref.set_index('GEOGRAPHY_CODE',inplace=True)


p_ref.to_sql('area_'+areatype,conn,if_exists='replace')
geomap = dict(zip(p_ref['AREACD'],p_ref.index))

p_ref

http://www.nomisweb.co.uk/api/v01/dataset/NM_1634_1.data.csv?date=latest&geography=TYPE464&cell=1,2&measures=20100&select=geography_code,date_name,cell,obs_value&uid=0x3cfb19ead752b37bb90da0eb3a0fe78baa9fa055
http://www.nomisweb.co.uk/api/v01/dataset/NM_144_1.data.csv?date=latest&geography=TYPE464&cell=1,2&measures=20100&select=geography_code,date_name,cell,obs_value&uid=0x3cfb19ead752b37bb90da0eb3a0fe78baa9fa055


,AREACD
GEOGRAPHY_CODE,
0,E06000001
1,E06000002
2,E06000003
3,E06000004
4,E06000005
...,...
343,W06000020
344,W06000021
345,W06000022


##  A function to get the individual dataframes

In [109]:
def process_df(url, src):
    # print(url)
    dummydf = pd.read_csv(url)
    dummydf.columns = 'GEOGRAPHY_CODE YEAR CELL OBS_VALUE'.split()

    if 'cellmaps' in src:
        cells = '0,'+','.join(src['cellnames'])
        cells = ','.join(src['cellnames'])
        for v, k in src['cellmaps'].items():
            for c in k:
                cells = cells.replace(c,v)

        src['cellnames'] = ('Totals,'+cells).split(',')

    mapping = dict(zip(eval(src['cells']), src['cellnames']))

    dummydf['CELL'] = dummydf['CELL'].apply(lambda x: mapping[x])   # (rename,mapping = mapping)

    return dummydf



## Process each Year and concat

In [128]:
def fetch(dataset):
    sources = dataset['sources']
    #     src = sources[0].copy()
    #      dont use pointers copy

    df = False
    for src in sources:
        src = src.copy()
        src['cells'] = ','.join(map(str, src['cells']))
        src['selection'] = ','.join(map(str, src['selection']))
        src['key'] = conf['apikey']
        if not hasattr(src, 'query_misc'):
            src['query_misc'] = ''

        url = conf['apiurl'].format(**src)
#         print(url)
        if type(df) != bool:
            df = pd.concat([df, process_df(url, src)])# df.append(process_df(url,src))
        else:
            df = process_df(url, src)


    # subcategory = mapping
    # print(df.columns)
    df = df[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE','YEAR']]
    df = df.sort_values('CELL')
    order = df.CELL.drop_duplicates().to_list()


    if 'Totals' not in order: 
        b = df.groupby('GEOGRAPHY_CODE').sum()
        b.CELL = 'Totals'
        df = pd.concat([df,b.reset_index()],axis=0)  
        order=['Totals']+order

    # else:
    #     order.remove('Totals')
    # # guarantee that totals is first and referenced as '0'
    # order = ['Totals'] + order
    mapping = dict(zip(order, range(len(order))))
    

    # datamong
    df['GEOGRAPHY_CODE'] = df['GEOGRAPHY_CODE'].apply(lambda x: geomap[x]).astype(np.uint8)
    i = df.groupby(['GEOGRAPHY_CODE', 'CELL']).sum().astype(np.uint32).reset_index()
    i.CELL = [mapping[k] for k in i.CELL]
    

    return [dataset['code'], list(enumerate(order)), i[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE']]]


### Parallel Fetch 


In [129]:
results = p_map(fetch,DATASETS)

100%|██████████| 7/7 [00:28<00:00,  4.04s/it]


### Write To DB

In [131]:
for name,order,df in results:
    print(name)
#     df.set_index('GEOGRAPHY_CODE',inplace=True)
    df.to_sql(name, con=conn, if_exists='replace')
    
    order = pd.DataFrame(order,columns = ['CELL','name']).set_index('CELL')
    order.to_sql(name+'_names', con=conn, if_exists='replace',index_label='CELL')


population
age
ethnicity
health
economic
travel
tenure


### SRINK THE DATABASE

In [133]:
from pathlib import Path
file_size =Path(db_file).stat().st_size

print("The file size WAS:", file_size/1048576,"MB")

conn.execute("VACUUM") ### SHRINK!!!!
conn.close()

file_size =Path(db_file).stat().st_size

print("The file size is:", file_size/1048576,"MB")

The file size WAS: 0.9453125 MB


ProgrammingError: Cannot operate on a closed database.